# Shor's Algorithm on Tket

<u><b>GOAL:</b></u> Find a number $p$ that divides $N$. <br>
<u><b>INPUT:</b></u> A positive integer $N$. <br>
<u><b>OUTPUT:</b></u> A positive integer $p$ that divides $N$.

## Classical Part

1. Pick a random number $a < N$ 
2. Find the greatest common divisor of $a$ and $N$.
> The Greatest Common Divisor (GCD) of two numbers is the largest positive integer that divides both of them. $GCD(a,b)$ can be found very fast with the Euclidean algorithm in $\mathcal{O}(log(min(a,b)))$.
3. If $GCD(a,N) \neq 1$ then we output $p=a$. Otherwise, run period finding subroutine to obtain a period $r$.
> $r$ is the smallest integer for which $f(x + r) = f(x)$, where $f(x) = a^x mod N$. The idea is that we want to find $r$ because: 

$$(a^r -1) mod N = 0.\$$

4. Then factors of $N$ should be $gcd(a^\frac{r}{2} ± 1, N)$.

> However, if $r$ is odd we need to go back to step 1 because since $a$ is also odd, then $ar$ will be odd and we will not get an integer when performing the division $\frac{ar}{2}$ on the last step.

> Also, since $(a^r-1) mod N = 0$, this imples that $a^r-1$ can be a any multiple of N, and therefore we also have to check that $a^{r/2} ≡ -1 (mod N)$. If this is the case then go back to step 1. 

## Quantum Part

> TODO: Talk about period finding subroutine